In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import HGate
from qiskit.circuit.library.generalized_gates import MCMTGate, DiagonalGate
from qiskit_aer import StatevectorSimulator
import numpy as np

In [51]:
def get_ctrl_qubits(binary_str):
    length = len(binary_str)
    for i, bit in enumerate(binary_str):
        if i == length - 1:
            raise ValueError("Invalid binary representation")
        if bit == '1':
            return (list(range(length - i - 1)), length - i - 1, binary_str[i+1:])
    raise ValueError("Invalid binary representation")

In [101]:
def get_renorm_coeff(num_velocities):
    num_qubits = int(np.ceil(np.log2(num_velocities)))
    floor_qubits = int(np.floor(np.log2(num_velocities)))
    full_velocities = 2**floor_qubits

    coeffs = [np.sqrt(2)**floor_qubits for _ in range(num_velocities)]
    for i in range(num_velocities - full_velocities):
        coeffs[i] *= np.sqrt(2)
        coeffs[-i - 1] *= np.sqrt(2)

    return coeffs

In [151]:
def test_gate(value):
    qc = QuantumCircuit(1)
    qc.p(value, 0)
    qc.x(0)
    qc.p(value, 0)
    qc.x(0)

    return qc.to_gate()


In [ ]:
num_velocities = 4
num_qubits = int(np.ceil(np.log2(num_velocities)))
floor_qubits = int(np.floor(np.log2(num_velocities)))
full_velocities = 2**floor_qubits

qc = QuantumCircuit(num_qubits + 1)
ancilla = num_qubits

for i in range(floor_qubits):
  qc.h(i)

for i in range(full_velocities, num_velocities):
   bin_repr = np.binary_repr(i, num_qubits)
   (ctrl_qubits, target_qubit, state) = get_ctrl_qubits(bin_repr)
   qc.append(HGate().control(num_ctrl_qubits=len(state), ctrl_state=state), ctrl_qubits + [target_qubit])

# Import and set up simulator
simulator = StatevectorSimulator()

qc = transpile(qc, simulator)

job = simulator.run(qc)
result = job.result()
statevector = result.get_statevector()
statevector.draw('latex')

<IPython.core.display.Latex object>

In [148]:

qc = QuantumCircuit(3)

qc.h(0)

qc.h(1)
qc.append(test_gate(0.7).control(ctrl_state='0'), [1,0])
qc.append(test_gate(-0.7).control(ctrl_state='0'), [1,0])
qc.h(1)
# Import and set up simulator
simulator = StatevectorSimulator()

qc = transpile(qc, simulator)

job = simulator.run(qc)
result = job.result()
statevector = result.get_statevector()
statevector.draw('latex')

<IPython.core.display.Latex object>